In [1]:


# from kaggle.api.kaggle_api_extended import KaggleApi
# import kagglehub
# import kaggle
# Initiliaze API




In [2]:

from sklearn import *
import pandas as pd
import seaborn as sns
# Donwload Dataset via API unzipped
# api.dataset_download_files(dataset="prathamtripathi/drug-classification", unzip=True)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
lol_df = pd.read_csv("Challenger_Ranked_Games_15minute.csv")
lol_df

,gameId,blueWins,blueTotalGolds,blueCurrentGolds,blueTotalLevel,blueAvgLevel,blueTotalMinionKills,blueTotalJungleMinionKills,blueFirstBlood,blueKill,...,redFirstTowerLane,redTowerKills,redMidTowerKills,redTopTowerKills,redBotTowerKills,redInhibitor,redFirstDragon,redDragnoType,redDragon,redRiftHeralds
0,4247263043,0,24081,1190,44,8.8,309,74,0,8,...,['MID_LANE'],2,1,0,1,0,1,"['WATER_DRAGON', 'EARTH_DRAGON']",2,1
1,4247155821,1,24162,2212,46,9.2,393,64,0,5,...,['TOP_LANE'],1,0,1,0,0,0,[],0,0
2,4243963257,0,22413,1563,41,8.2,300,62,0,5,...,['BOT_LANE'],4,2,1,1,0,1,"['FIRE_DRAGON', 'EARTH_DRAGON']",2,1
3,4241678498,0,23837,3197,46,9.2,370,96,0,6,...,['TOP_LANE'],1,0,1,0,0,0,[],0,1
4,4241538868,1,27688,3663,44,8.8,381,66,0,9,...,['BOT_LANE'],2,1,0,1,0,0,[],0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26829,4143231833,0,26110,1535,46,9.2,289,73,0,15,...,['BOT_LANE'],1,0,0,1,0,1,['WATER_DRAGON'],1,1
26830,4157911901,0,57503,3293,62,12.4,329,28,0,27,...,[],6,2,1,3,1,0,[],0,0
26831,3764171638,0,26091,2986,47,9.4,338,86,0,7,...,[],0,0,0,0,0,0,[],0,0
26832,4110201724,1,24734,4289,45,9.0,328,64,0,11,...,['BOT_LANE'],1,0,0,1,0,0,['AIR_DRAGON'],1,0


In [4]:
# Count the number of differences
difference_count = (lol_df['blueWins'] != lol_df['redWins']).sum()

print(f"Number of differences: {difference_count}")


Number of differences: 26834


lol_df = lol_df.drop(columns=['redDragnoType', 'blueDragnoType', 'redFirstBlood', 'blueFirstBlood'])
lol_df = lol_df.drop_duplicates()

In [5]:
lol_df = lol_df.drop(columns=['redFirstBlood', 'blueFirstBlood'])
lol_df = lol_df.drop_duplicates()

In [6]:
lol_df = pd.get_dummies(lol_df)

In [7]:
lol_corr = lol_df.corr()
lol_corr

,gameId,blueWins,blueTotalGolds,blueCurrentGolds,blueTotalLevel,blueAvgLevel,blueTotalMinionKills,blueTotalJungleMinionKills,blueKill,blueDeath,...,"redDragnoType_['WATER_DRAGON', 'AIR_DRAGON', 'FIRE_DRAGON']","redDragnoType_['WATER_DRAGON', 'AIR_DRAGON']","redDragnoType_['WATER_DRAGON', 'EARTH_DRAGON', 'AIR_DRAGON', 'AIR_DRAGON']","redDragnoType_['WATER_DRAGON', 'EARTH_DRAGON']","redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON', 'AIR_DRAGON']","redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON', 'EARTH_DRAGON']","redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON']","redDragnoType_['WATER_DRAGON', 'WATER_DRAGON']",redDragnoType_['WATER_DRAGON'],redDragnoType_[]
gameId,1.000000,-0.005397,0.001830,-0.009425,0.040736,0.040736,-0.076390,-0.041898,0.001016,-0.003449,...,-0.001389,0.008220,-0.002263,0.002762,-0.004503,-0.006312,-0.006266,-0.024747,-0.003234,0.016960
blueWins,-0.005397,1.000000,0.225555,0.261220,0.162420,0.162420,0.160936,0.137863,0.279579,-0.257865,...,-0.008202,-0.072166,-0.006112,-0.069009,-0.006112,-0.006114,-0.081301,0.010583,-0.047865,0.296870
blueTotalGolds,0.001830,0.225555,1.000000,0.361546,0.919173,0.919173,-0.203182,-0.378317,0.917561,0.643619,...,0.051194,-0.043565,0.025898,-0.039175,0.027971,0.046982,-0.040577,0.017233,-0.074038,0.274931
blueCurrentGolds,-0.009425,0.261220,0.361546,1.000000,0.303369,0.303369,-0.044676,-0.099753,0.377635,0.036493,...,0.006676,-0.039992,0.013962,-0.036403,0.000323,0.001936,-0.024224,0.002096,-0.045992,0.190912
blueTotalLevel,0.040736,0.162420,0.919173,0.303369,1.000000,1.000000,-0.178894,-0.388435,0.800904,0.614591,...,0.033329,-0.036256,0.015903,-0.031122,0.017331,0.021814,-0.034913,0.010237,-0.071560,0.251489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON', 'EARTH_DRAGON']",-0.006312,-0.006114,0.046982,0.001936,0.021814,0.021814,-0.001128,-0.008996,0.032400,0.050582,...,-0.000167,-0.001642,-0.000075,-0.001567,-0.000075,1.000000,-0.001582,-0.000129,-0.003982,-0.010131
"redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON']",-0.006266,-0.081301,-0.040577,-0.024224,-0.034913,-0.034913,0.016919,0.006147,-0.052132,0.015902,...,-0.001768,-0.017389,-0.000791,-0.016589,-0.000791,-0.001582,1.000000,-0.001370,-0.042151,-0.107258
"redDragnoType_['WATER_DRAGON', 'WATER_DRAGON']",-0.024747,0.010583,0.017233,0.002096,0.010237,0.010237,-0.005391,-0.005404,0.019032,0.019356,...,-0.000145,-0.001422,-0.000065,-0.001357,-0.000065,-0.000129,-0.001370,1.000000,-0.003448,-0.008774
redDragnoType_['WATER_DRAGON'],-0.003234,-0.047865,-0.074038,-0.045992,-0.071560,-0.071560,0.061365,0.057455,-0.081393,-0.033624,...,-0.004452,-0.043774,-0.001991,-0.041761,-0.001991,-0.003982,-0.042151,-0.003448,1.000000,-0.270015


high_corr_pairs = []

# Iterate through the correlation matrix
for i in range(len(lol_corr.columns)):
    for j in range(i+1, len(lol_corr.columns)):  # Iterate only the upper triangle to avoid duplicates
        if abs(lol_corr.iloc[i, j]) > 0.7:  # Check correlation value
            high_corr_pairs.append([lol_corr.index[i], lol_corr.columns[j]])

In [8]:
# Threshold to consider highly correlated features
high_threshold = 0.8

# Find list pairings of highly correlated features (excluding 'blueWins')
high_corr_pairs = []
for i in range(len(lol_corr.columns)):
    for j in range(i + 1, len(lol_corr.columns)):
        if abs(lol_corr.iloc[i, j]) > high_threshold and lol_corr.index[i] != 'blueWins' and lol_corr.columns[j] != 'blueWins':
            high_corr_pairs.append([lol_corr.index[i], lol_corr.columns[j]])

# Compare correlations with 'blueWins' and determine which features to drop
features_to_drop = []
for pair in high_corr_pairs:
    feature1, feature2 = pair
    if abs(lol_corr.loc[feature1, 'blueWins']) >= abs(lol_corr.loc[feature2, 'blueWins']):
        features_to_drop.append(feature2)  # Drop the feature with lower correlation with blueWins
    else:
        features_to_drop.append(feature1)

# Remove duplicates from the features_to_drop list
features_to_drop = list(set(features_to_drop))

# Output the features to drop
features_to_drop

['blueFirstDragon',
 'redAssist',
 'redDragnoType_[]',
 'redInhibitor',
 'redDeath',
 'redFirstTowerLane_[]',
 'blueTotalGolds',
 'redFirstDragon',
 'blueInhibitor',
 'blueDragnoType_[]',
 'blueAvgLevel',
 'redAvgLevel',
 'blueFirstTowerLane_[]',
 'redKill',
 'redTotalLevel',
 'blueTotalLevel',
 'blueAssist',
 'redFirstTower',
 'redTotalGolds']

In [17]:
lol_df_dropped = lol_df.drop(columns=features_to_drop)
lol_df_dropped = lol_df_dropped.drop(columns="gameId")
lol_df_dropped

,blueWins,blueCurrentGolds,blueTotalMinionKills,blueTotalJungleMinionKills,blueKill,blueDeath,blueWardPlaced,blueWardKills,blueFirstTower,blueFirstInhibitor,...,"redDragnoType_['WATER_DRAGON', 'AIR_DRAGON', 'FIRE_DRAGON', 'FIRE_DRAGON']","redDragnoType_['WATER_DRAGON', 'AIR_DRAGON', 'FIRE_DRAGON']","redDragnoType_['WATER_DRAGON', 'AIR_DRAGON']","redDragnoType_['WATER_DRAGON', 'EARTH_DRAGON', 'AIR_DRAGON', 'AIR_DRAGON']","redDragnoType_['WATER_DRAGON', 'EARTH_DRAGON']","redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON', 'AIR_DRAGON']","redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON', 'EARTH_DRAGON']","redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON']","redDragnoType_['WATER_DRAGON', 'WATER_DRAGON']",redDragnoType_['WATER_DRAGON']
0,0,1190,309,74,8,14,29,8,0,0,...,False,False,False,False,True,False,False,False,False,False
1,1,2212,393,64,5,6,31,6,0,0,...,False,False,False,False,False,False,False,False,False,False
2,0,1563,300,62,5,20,28,7,0,0,...,False,False,False,False,False,False,False,False,False,False
3,0,3197,370,96,6,13,108,9,0,0,...,False,False,False,False,False,False,False,False,False,False
4,1,3663,381,66,9,10,36,11,0,0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26829,0,1535,289,73,15,14,38,9,0,0,...,False,False,False,False,False,False,False,False,False,True
26830,0,3293,329,28,27,37,36,2,1,0,...,False,False,False,False,False,False,False,False,False,False
26831,0,2986,338,86,7,6,40,12,1,0,...,False,False,False,False,False,False,False,False,False,False
26832,1,4289,328,64,11,7,33,10,0,0,...,False,False,False,False,False,False,False,False,False,False


In [10]:
#wins target

In [11]:
lol_df

,gameId,blueWins,blueTotalGolds,blueCurrentGolds,blueTotalLevel,blueAvgLevel,blueTotalMinionKills,blueTotalJungleMinionKills,blueKill,blueDeath,...,"redDragnoType_['WATER_DRAGON', 'AIR_DRAGON', 'FIRE_DRAGON']","redDragnoType_['WATER_DRAGON', 'AIR_DRAGON']","redDragnoType_['WATER_DRAGON', 'EARTH_DRAGON', 'AIR_DRAGON', 'AIR_DRAGON']","redDragnoType_['WATER_DRAGON', 'EARTH_DRAGON']","redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON', 'AIR_DRAGON']","redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON', 'EARTH_DRAGON']","redDragnoType_['WATER_DRAGON', 'FIRE_DRAGON']","redDragnoType_['WATER_DRAGON', 'WATER_DRAGON']",redDragnoType_['WATER_DRAGON'],redDragnoType_[]
0,4247263043,0,24081,1190,44,8.8,309,74,8,14,...,False,False,False,True,False,False,False,False,False,False
1,4247155821,1,24162,2212,46,9.2,393,64,5,6,...,False,False,False,False,False,False,False,False,False,True
2,4243963257,0,22413,1563,41,8.2,300,62,5,20,...,False,False,False,False,False,False,False,False,False,False
3,4241678498,0,23837,3197,46,9.2,370,96,6,13,...,False,False,False,False,False,False,False,False,False,True
4,4241538868,1,27688,3663,44,8.8,381,66,9,10,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26829,4143231833,0,26110,1535,46,9.2,289,73,15,14,...,False,False,False,False,False,False,False,False,True,False
26830,4157911901,0,57503,3293,62,12.4,329,28,27,37,...,False,False,False,False,False,False,False,False,False,True
26831,3764171638,0,26091,2986,47,9.4,338,86,7,6,...,False,False,False,False,False,False,False,False,False,True
26832,4110201724,1,24734,4289,45,9.0,328,64,11,7,...,False,False,False,False,False,False,False,False,False,False
